# DIMS.FARM REST API example

In [9]:
import os
import json

import pandas as pd
from requests import HTTPError

from watobs.datafarm import DatafarmRepository

api_key = os.getenv("DATAFARM_API_KEY")
assert api_key is not None


### Connect to DIMS.FARM REST API

In [10]:
dfr = DatafarmRepository(api_key)
dfr.connect()

### List available time series

In [11]:
time_series_list = dfr.list_time_series()
time_series_list.head()

,ID,EntityID,Touched,IDName,IDDescription,LocationID,TimeSeriesParameterID,TimeSeriesMediaID,TimeSeriesUnitID,TimeSeriesDataTypeID,TimeSeriesDatasourceID,TimeSeriesTypeID,TimeSeriesStatusID,TimeSeriesDataProviderID,TimeSeriesDataProviderArgument,DataExpectedCount,DataArchiveAfter,DataDeleteAfter
GUID,,,,,,,,,,,,,,,,,,
{62F60AF2-C34A-11ED-B2F7-1831BF2DC749},2,AKZ_waves_CMEMS_unfiltered_Hm0,2023-03-20 17:18:42,AKZ_waves_CMEMS_unfiltered_Hm0,Spectral significant wave height,AKZ,Hm0,waves,m,Provider,,CMEMS,unfiltered,,,NaN,NaN,NaN
{7394526A-C34A-11ED-B2F7-1831BF2DC749},3,AKZ_waves_CMEMS_unfiltered_Tz,2023-03-20 17:18:43,AKZ_waves_CMEMS_unfiltered_Tz,Average zero crossing wave period (Tz),AKZ,Tz,waves,s,Provider,,CMEMS,unfiltered,,,NaN,NaN,NaN
{7394526B-C34A-11ED-B2F7-1831BF2DC749},4,Bor1_currents_RVO-FUGRO_derived_CD,2023-03-20 17:18:43,Bor1_currents_RVO-FUGRO_derived_CD,Current direction,Bor1,CD,currents,degree-N-to,Provider,,RVO-FUGRO,derived,,,NaN,NaN,NaN
{7394526C-C34A-11ED-B2F7-1831BF2DC749},5,Bor1_currents_RVO-FUGRO_derived_CS,2023-03-20 17:18:44,Bor1_currents_RVO-FUGRO_derived_CS,Current speed,Bor1,CS,currents,m/s,Provider,,RVO-FUGRO,derived,,,NaN,NaN,NaN
{7394526D-C34A-11ED-B2F7-1831BF2DC749},6,Bor1_currents_RVO-FUGRO_missing_CD-4,2023-03-20 17:18:45,Bor1_currents_RVO-FUGRO_missing_CD-4,Current direction,Bor1,CD,currents,degree-N-to,Provider,,RVO-FUGRO,missing,,,NaN,NaN,NaN


### Get time series data

In [22]:
data = dfr.get_data(
    time_series_id="testapi.insert",
    start="2015-03-24T10:16:45.034Z",
    end="2023-03-24T10:16:45.034Z",
    limit=5,
    ascending=True
)
data

,Data,QualityTxt
RefDateTimeRef,,
2022-09-28 11:02:36,637.0,ok
2022-09-28 11:02:36,637.0,ok
2022-09-28 11:02:41,637.0,ok
2022-09-28 11:32:34,637.0,ok
2022-09-28 11:32:34,637.0,ok


### Insert time series data

Here we prepare some data to insert in form of a dataframe.

In [23]:
new_data = data.head(2).copy()
new_data["Data"] = new_data["Data"] + 1
new_data["TimeStamp"] = new_data.index + pd.Timedelta("3650 days")
new_data["QualityLevel"] = new_data["QualityTxt"]
new_data = new_data.drop(columns=["QualityTxt"])
new_data.iloc[0,0] = None
new_data


,Data,TimeStamp,QualityLevel
RefDateTimeRef,,,
2022-09-28 11:02:36,NaN,2032-09-25 11:02:36,ok
2022-09-28 11:02:36,638.0,2032-09-25 11:02:36,ok


We can peek at the json body for the call to the API. The NaN value is represented as {"N": 1, "V": 0}.

In [24]:
body = dfr._get_insert_data_body("testapi.insert", new_data)
body_json = json.dumps(body, indent=2)
print(body_json)

{
  "BulkInsert": false,
  "TimeSeriesName": "testapi.insert",
  "Data": [
    {
      "N": 1,
      "V": 0.0
    },
    {
      "N": 0,
      "V": 638.0
    }
  ],
  "TimeStamp": [
    "2032-09-25T11:02:36",
    "2032-09-25T11:02:36"
  ],
  "QualityLevel": [
    "ok",
    "ok"
  ]
}


To insert the data we use the `insert_data` method:

In [25]:
try:
    dfr.insert_data("testapi.insert", new_data)  # TODO: Error 500 ??
except HTTPError as e:
    print(e)

500 Server Error: Internal Server Error for url: https://apidevtest.datafarm.work/api/TimeSeries/InsertData


### Close connection

In [26]:
dfr.close()

# Now this will fail:
try:
    dfr.list_time_series()
except HTTPError as e:
    print(e)

401 Client Error: Unauthorized for url: https://apidevtest.datafarm.work/api/List/TimeSeries/


In [27]:
# Reconnect and try again
dfr.connect()
dfr.list_time_series().head()
dfr.close()

,ID,EntityID,Touched,IDName,IDDescription,LocationID,TimeSeriesParameterID,TimeSeriesMediaID,TimeSeriesUnitID,TimeSeriesDataTypeID,TimeSeriesDatasourceID,TimeSeriesTypeID,TimeSeriesStatusID,TimeSeriesDataProviderID,TimeSeriesDataProviderArgument,DataExpectedCount,DataArchiveAfter,DataDeleteAfter
GUID,,,,,,,,,,,,,,,,,,
{62F60AF2-C34A-11ED-B2F7-1831BF2DC749},2,AKZ_waves_CMEMS_unfiltered_Hm0,2023-03-20 17:18:42,AKZ_waves_CMEMS_unfiltered_Hm0,Spectral significant wave height,AKZ,Hm0,waves,m,Provider,,CMEMS,unfiltered,,,NaN,NaN,NaN
{7394526A-C34A-11ED-B2F7-1831BF2DC749},3,AKZ_waves_CMEMS_unfiltered_Tz,2023-03-20 17:18:43,AKZ_waves_CMEMS_unfiltered_Tz,Average zero crossing wave period (Tz),AKZ,Tz,waves,s,Provider,,CMEMS,unfiltered,,,NaN,NaN,NaN
{7394526B-C34A-11ED-B2F7-1831BF2DC749},4,Bor1_currents_RVO-FUGRO_derived_CD,2023-03-20 17:18:43,Bor1_currents_RVO-FUGRO_derived_CD,Current direction,Bor1,CD,currents,degree-N-to,Provider,,RVO-FUGRO,derived,,,NaN,NaN,NaN
{7394526C-C34A-11ED-B2F7-1831BF2DC749},5,Bor1_currents_RVO-FUGRO_derived_CS,2023-03-20 17:18:44,Bor1_currents_RVO-FUGRO_derived_CS,Current speed,Bor1,CS,currents,m/s,Provider,,RVO-FUGRO,derived,,,NaN,NaN,NaN
{7394526D-C34A-11ED-B2F7-1831BF2DC749},6,Bor1_currents_RVO-FUGRO_missing_CD-4,2023-03-20 17:18:45,Bor1_currents_RVO-FUGRO_missing_CD-4,Current direction,Bor1,CD,currents,degree-N-to,Provider,,RVO-FUGRO,missing,,,NaN,NaN,NaN


### Using the context manager

In [32]:

with DatafarmRepository(api_key) as dfr:
    data = dfr.get_data(
        time_series_id="testapi.insert",
        limit=2
    )
data

,Data,QualityTxt
RefDateTimeRef,,
2022-09-28 11:02:36,637.0,ok
2022-09-28 11:02:36,637.0,ok
